In [27]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('/content/Ecommerce_Data-1.csv')
df.head()


,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,439570,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12
1,387281,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10
2,337863,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9
3,57628,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13
4,330897,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10


In [28]:
df.shape

(81601, 10)

In [29]:
df['total_revenue']= df['Quantity']*df['UnitPrice']
df.drop('Unnamed: 0', axis=1, inplace=True)
df['Date']=pd.to_datetime(df['Date'])
df= df[df['Quantity']>0]
df.head()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,total_revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9,45.00
3,541215,22662,LUNCH BAG DOLLY GIRL DESIGN,10,1.65,14329.0,United Kingdom,2011-01-14,13,16.50
4,565930,POST,POSTAGE,5,18.00,12685.0,France,2011-09-08,10,90.00


In [30]:
max_date = df['Date'].max()
df2 = df.groupby('CustomerID').agg(
    { 'Date': lambda x: (max_date-x.min()).days,
      'Quantity': lambda x: x.sum(),
      'total_revenue': lambda x: x.sum(),
      'InvoiceNo': lambda x: x.sum(),

    }
)
df2

,Date,Quantity,total_revenue,InvoiceNo
CustomerID,,,,
12347.0,367,623,1146.59,5735115422375422375620325735115735115735115562...
12348.0,358,531,550.92,539318568172539318548955541998539318548955
12349.0,18,203,406.32,5776095776095776095776095776095776095776095776...
12350.0,310,36,45.60,543037543037
12352.0,296,163,663.33,5686995453325675055473905686995675055742755742...
...,...,...,...,...
18280.0,277,9,52.75,545712545712545712
18281.0,180,11,33.45,556464556464
18282.0,126,11,67.85,580173562525580173580173


In [31]:
max_date

Timestamp('2011-12-09 00:00:00')

In [32]:
def customer_model(data):
    max_date = data['Date'].max()
    data = data.groupby('CustomerID').agg(
      {
      'Date': lambda x: (max_date-x.min()).days,
      'Quantity': lambda x: x.sum(),
      'total_revenue': lambda x: x.sum(),
      'InvoiceNo': lambda x: len(x),
      })
    return data

df['Date'] = pd.to_datetime(df['Date']) # assuming 'Date' column contains date information
data = customer_model(df)
data.head()

,Date,Quantity,total_revenue,InvoiceNo
CustomerID,,,,
12347.0,367,623,1146.59,48
12348.0,358,531,550.92,7
12349.0,18,203,406.32,16
12350.0,310,36,45.60,2
12352.0,296,163,663.33,21


In [33]:
data.columns = ['age','quantity','total_rev','num_trans']
data = data[data['quantity']>0]
data.head()

,age,quantity,total_rev,num_trans
CustomerID,,,,
12347.0,367,623,1146.59,48
12348.0,358,531,550.92,7
12349.0,18,203,406.32,16
12350.0,310,36,45.60,2
12352.0,296,163,663.33,21


**GETTING THE ELEMENTS OF CLTV** **CALCULATION**

In [34]:
data['avg_order_value'] = data['total_rev']/data['num_trans']
data

,age,quantity,total_rev,num_trans,avg_order_value
CustomerID,,,,,
12347.0,367,623,1146.59,48,23.887292
12348.0,358,531,550.92,7,78.702857
12349.0,18,203,406.32,16,25.395000
12350.0,310,36,45.60,2,22.800000
12352.0,296,163,663.33,21,31.587143
...,...,...,...,...,...
18280.0,277,9,52.75,3,17.583333
18281.0,180,11,33.45,2,16.725000
18282.0,126,11,67.85,4,16.962500


In [35]:
purchase_freq = sum(data['num_trans'])/len(data)
purchase_freq

19.496336101612115

In [36]:
repeat_rate = data[data['num_trans']>1].shape[0]/data.shape[0]
repeat_rate

0.9291646311675623

In [37]:
churn_rate = 1-repeat_rate
churn_rate

0.07083536883243768

In [38]:
data['profit_margin'] = data['total_rev']*10

In [39]:
data.head()

,age,quantity,total_rev,num_trans,avg_order_value,profit_margin
CustomerID,,,,,,
12347.0,367,623,1146.59,48,23.887292,11465.9
12348.0,358,531,550.92,7,78.702857,5509.2
12349.0,18,203,406.32,16,25.395000,4063.2
12350.0,310,36,45.60,2,22.800000,456.0
12352.0,296,163,663.33,21,31.587143,6633.3


**PUTTING ALL THE EQUATIONS TOGETHER WE GET**

In [40]:
data['CLTV'] = ((data['avg_order_value']*purchase_freq)/churn_rate)*.10
data

,age,quantity,total_rev,num_trans,avg_order_value,profit_margin,CLTV
CustomerID,,,,,,,
12347.0,367,623,1146.59,48,23.887292,11465.9,657.460637
12348.0,358,531,550.92,7,78.702857,5509.2,2166.174018
12349.0,18,203,406.32,16,25.395000,4063.2,698.957969
12350.0,310,36,45.60,2,22.800000,456.0,627.534621
12352.0,296,163,663.33,21,31.587143,6633.3,869.387093
...,...,...,...,...,...,...,...
18280.0,277,9,52.75,3,17.583333,527.5,483.953966
18281.0,180,11,33.45,2,16.725000,334.5,460.329672
18282.0,126,11,67.85,4,16.962500,678.5,466.866491


In [41]:
import seaborn as sns
print("the median CLTV is", data['CLTV'].median())
print("the average CLTV is", data['CLTV'].mean())

the median CLTV is 460.5410131773401
the average CLTV is 747.7434697480141


In [42]:
data.sort_values(by='CLTV').reset_index()

,CustomerID,age,quantity,total_rev,num_trans,avg_order_value,profit_margin,CLTV
0,12977.0,156,2,0.78,1,0.780000,7.8,21.468290
1,14661.0,26,1,0.85,1,0.850000,8.5,23.394931
2,16658.0,368,1,0.85,1,0.850000,8.5,23.394931
3,17962.0,142,3,2.99,3,0.996667,29.9,27.431703
4,17859.0,304,5,5.21,5,1.042000,52.1,28.679433
...,...,...,...,...,...,...,...,...
4089,12931.0,239,5590,9344.82,15,622.988000,93448.2,17146.777994
4090,16333.0,298,7840,7944.40,11,722.218182,79444.0,19877.934771
4091,17949.0,289,4565,7671.45,10,767.145000,76714.5,21114.475728
4092,16000.0,2,2630,7948.30,5,1589.660000,79483.0,43752.924786


In [43]:
data.to_csv("finalEC_dataset.csv")

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [45]:
X = data.drop('CLTV',axis=1)
y = data['CLTV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [46]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [47]:
model = xgb.XGBRegressor()
model.fit(X_train_scaled, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [48]:
y_pred = model.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", rmse)


Root Mean Squared Error: 309.94906920106325


In [49]:
new_customer_data = pd.read_csv('finalEC_dataset.csv')


In [50]:
print(X_train.shape)
print(y_train.shape)

(3275, 6)
(3275,)


In [51]:
!pip install lifetimes

In [52]:
import pandas as pd
from lifetimes import GammaGammaFitter
from lifetimes import BetaGeoFitter
from lifetimes.utils import summary_data_from_transaction_data

In [53]:
data


,age,quantity,total_rev,num_trans,avg_order_value,profit_margin,CLTV
CustomerID,,,,,,,
12347.0,367,623,1146.59,48,23.887292,11465.9,657.460637
12348.0,358,531,550.92,7,78.702857,5509.2,2166.174018
12349.0,18,203,406.32,16,25.395000,4063.2,698.957969
12350.0,310,36,45.60,2,22.800000,456.0,627.534621
12352.0,296,163,663.33,21,31.587143,6633.3,869.387093
...,...,...,...,...,...,...,...
18280.0,277,9,52.75,3,17.583333,527.5,483.953966
18281.0,180,11,33.45,2,16.725000,334.5,460.329672
18282.0,126,11,67.85,4,16.962500,678.5,466.866491
